In [ ]:
# dependencies
import pymongo as pm
import urllib
from scrape import Scrape as sc
from pprint import pprint
from bs4 import BeautifulSoup as bs
import re
import time

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn)

In [ ]:
# define db and collections
db = client.scrape_db

# this collection will store all the HTML pages
htmlCollection = db.HTML_pages

In [4]:
# startURL = 'http://books.toscrape.com/'
# domain = 'toscrape.com'

# declare top-level url and domain
# startURL = 'https://billypenn.com/'
# domain = 'billypenn.com'

# startURL = 'https://theintercept.com/'
# domain = 'theintercept.com'

# startURL = 'https://www.wsj.com/'
# domain = 'wsj.com/'

# startURL = 'https://www.nationalreview.com/'
# domain = 'nationalreview.com/'

startURL = 'https://www.economist.com/'
domain = 'economist.com/'

# establish necessary data structures
done_urls = set()
external_urls = []
urls = set()
pagelinks = set()

# seed with top-level url of website
urls.add(startURL)

In [5]:
len(urls)

1

In [6]:
# counter for limiting functions
ctr = 0

# continue to loop through urls until we have scraped all valid
# needed a while loop so we can modify size of urls through process
while (len(urls) > 0) and ctr < 10:    #(len(done_urls) < 200):
    
    # timer for performance across url loops
    forStart = time.time()
    
    # go through every url that has not been scraped yet
    for url in urls:
        print("URL in for loop: ", url)
        
        # timer for performance
        start = time.time()
        
        # governor to avoid being banned/blocked
        time.sleep(5)   # time governor
        
        # check for email and run all other links through scrape
        html = sc.scrapeHTML(url)
        
        # end timer for performance and print status
        end = time.time()
        print(f'Elapsed time: {end - start}')
        
        # validate html and then add to the pagelinks all the links that were scraped
        if html: 
            #print(f'Html scraped: {html[:50]}')
            data = {
                "url": url, 
                "html": html, 
                }

            # insert dict into db document
            try:
                htmlCollection.insert_one(data)
            except Exception as e:
                print(f'htmlCollection Insert: {e}')
                done_urls.add(url)
                continue
            
            # record url in "done" collection
            done_urls.add(url)
            print(f'done_urls: {len(done_urls)}')
            
            pagelinks = pagelinks.union(sc.scrapeURL(html, url))
            print(f'Extracted links: {len(pagelinks)}')
        else:
            done_urls.add(url)
            print(f'Main Loop: HTML not extracted')
        
        ####################################################
        # TODO: update done_urls, external and urls to CSV #
        ####################################################
        
    # end timer of for loop
    forEnd = time.time()
    
    # print status for testing
    print()
    print(f'Number of Urls: {len(urls)}')
    print(f'Process Time: {forEnd -forStart}')
    print('--------------------------------------------------------------')
        
        
    
    # add links from page to urls set outside of for loop
    if html:
        for link in pagelinks:
            if domain in link:
                urls.add(link)
                # print(f"Link in pagelinks: {link}")
            else:
                external_urls.append(link)
    
    # remove done_urls from todo urls and increment counter
    urls -= done_urls
    ctr += 1
    
    # print status for testing
    print("done_urls: ", len(done_urls))
    print("(urls - done_urls): ", len(urls))
    print("LOOP counter: ", ctr)
    print("--------------------")


URL in for loop:  https://www.economist.com/
Elapsed time: 5.1374242305755615
done_urls: 1
Extracted links: 130

Number of Urls: 1
Process Time: 5.25428581237793
--------------------------------------------------------------
done_urls:  1
(urls - done_urls):  101
LOOP counter:  1
--------------------
URL in for loop:  https://www.economist.com/news/leaders/21744070-greatest-risk-pluralism-young-democracies-where-checks-and-balances-are-not-yet
Elapsed time: 5.153255462646484
done_urls: 2
Extracted links: 140
URL in for loop:  https://www.economist.com/free-email-newsletter-signup?destination=%2F
Elapsed time: 5.099889039993286
done_urls: 3
Extracted links: 144
URL in for loop:  https://www.economist.com/blogs/openfuture/2018/06/open-society-0
Elapsed time: 5.190498113632202
done_urls: 4
Extracted links: 158
URL in for loop:  https://www.economist.com/news/leaders/21744065-logic-irish-border-forcing-britain-towards-close-relationship-eu-britain
Elapsed time: 6.879143238067627
done_urls:

KeyboardInterrupt: 

In [ ]:
counter = 0
for url in urls:
    print(f'URL #{counter}: {url}')